<a href="https://colab.research.google.com/github/martians-sheep/pl_task_recomended_csd/blob/main/text_vectorization_and_worker_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# テキストのトークン数を確認する関数

In [ ]:
# トークナイザーのインストール
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.5 MB/s eta 0:00:00


In [ ]:
import tiktoken

def get_tokens(text):
  # エンコーディングの取得
  # embeddingで利用するモデルの"text-embedding-ada-002" の トークナイザーとして"cl100k_base"を利用する
  enc = tiktoken.get_encoding("cl100k_base")
  # エンコードの実行
  tokens = enc.encode(text)
  return tokens

In [ ]:
# get_tokens関数の実行
tokens = get_tokens("こんにちは、世界！")
print(len(tokens))

6


# サンプルデータのファイルパスを返す関数

In [ ]:
from google.colab import drive
import os

# Google Driveをマウントする
drive.mount('/content/drive')

# 起点となるフォルダのパスを指定する
base_folder = '/content/drive/MyDrive/{サンプルデータのフォルダ名}'

def get_file_paths(folder_path):
    file_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.txt'):  # テキストファイルのみ処理する
                file_path = os.path.join(root, file)
                file_paths.append(file_path)
    return file_paths

Mounted at /content/drive


In [ ]:
# get_file_paths関数の実行
file_paths = get_file_paths(base_folder)

NameError: name 'get_file_paths' is not defined

# ファイルパスを元にテキストのトークン数を確認する

In [ ]:
# ファイルパスからテキストを取得する関数
def get_file_content(file_path):
    try:
        with open(file_path, 'r') as file:
            content = file.read()
            return content
    except IOError:
        print(f"ファイル {file_path} の読み込み中にエラーが発生しました。")
        return None

In [ ]:
# ファイルパスのリストをトークン数制限に基づいてグループ化する関数
# 一度に扱えるtoken数が8000であるため8000毎にグループ化する
# todo　： クソデカファイルの扱い
def split_files_by_token_count(file_paths, max_tokens=8000):
    file_path_groups = []
    current_group = []
    current_token_count = 0

    for file_path in file_paths:
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
            token_count = len(get_tokens(text))

            if current_token_count + token_count <= max_tokens:
                current_group.append(file_path)
                current_token_count += token_count
            else:
                file_path_groups.append(current_group)
                current_group = [file_path]
                current_token_count = token_count

    if current_group:
        file_path_groups.append(current_group)

    return file_path_groups

In [ ]:
# split_files_by_token_count関数の実行
file_path_groups = split_files_by_token_count(file_paths)
# グループ数
print(len(file_path_groups))

# CSVファイルとして作業実施者と作業済みファイルのデータの作成

In [ ]:
import csv
import random

# 作業実施者と作業済みファイルを紐付けたCSVを作成する
# ここでは作業実施者は5名を想定
def assign_file_info(file_paths, csv_file):
    # CSVファイルが存在しない場合は新規作成し、ヘッダーを書き込む
    if not os.path.exists(csv_file):
        with open(csv_file, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['index', 'file_path', 'worker_id'])

    # 既存のCSVファイルからデータを読み込む
    existing_data = []
    if os.path.exists(csv_file):
        with open(csv_file, mode='r') as file:
            reader = csv.reader(file)
            next(reader)  # ヘッダーをスキップ
            existing_data = list(reader)

    # worker_idごとの優先ファイルパスの設定
    priority_paths = {
        1: 'IT',
        2: 'houmu',
        3: 'qa',
        4: 'quality',
        5: 'r_and_d'
    }

    # 新しいデータを生成
    new_data = []
    for file_path in file_paths:
        index = len(existing_data) + len(new_data)

        # 優先ファイルパスに基づいてworker_idを割り当て
        assigned_worker_id = None
        for worker_id, priority_path in priority_paths.items():
            if priority_path in file_path:
                if random.random() < 0.8:  # 80%の確率で優先割り当て
                    assigned_worker_id = worker_id
                    break

        # 優先割り当てされなかった場合はランダムにworker_idを割り当て
        if assigned_worker_id is None:
            assigned_worker_id = random.randint(1, 5)

        new_data.append([index, file_path, assigned_worker_id])

           # 新しいデータをCSVファイルに追記
    with open(csv_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(new_data)

    print(f"Assigned {len(new_data)} files to workers.")

In [ ]:
# CSVファイルのファイルパス
csv_file = "./worked_files.csv"
# CSVファイルの作成
assign_file_info(file_paths, csv_file)

NameError: name 'file_paths' is not defined

In [ ]:
# フォルダごとに割り当てられた担当者の割合を出す関数
from collections import defaultdict

def analyze_worker_distribution(csv_file):
    # フォルダごとの担当者の割合を格納する辞書
    folder_worker_distribution = {
        'IT': defaultdict(int),
        'houmu': defaultdict(int),
        'qa': defaultdict(int),
        'quality': defaultdict(int),
        'r_and_d': defaultdict(int)
    }

    # CSVファイルからデータを読み込む
    with open(csv_file, mode='r') as file:
        reader = csv.reader(file)
        next(reader)  # ヘッダーをスキップ

        # 各行を処理
        for row in reader:
            file_path = row[1]
            worker_id = int(row[2])

            # フォルダごとに担当者の割合を集計
            for folder in folder_worker_distribution.keys():
                if folder in file_path:
                    folder_worker_distribution[folder][worker_id] += 1
                    break

    # 割合を計算して表示
    for folder, worker_counts in folder_worker_distribution.items():
        total_files = sum(worker_counts.values())
        if total_files > 0:
            print(f"Folder: {folder}")
            for worker_id, count in worker_counts.items():
                percentage = count / total_files * 100
                print(f"  Worker {worker_id}: {percentage:.2f}%")
        else:
            print(f"Folder: {folder}")
            print("  No files assigned.")
        print()

In [ ]:
# フォルダ毎に割り当てられた人の割合を確認する
analyze_worker_distribution(csv_file)

NameError: name 'analyze_worker_distribution' is not defined

# テキストデータのベクトル化(Embedding)

In [ ]:
# OpenAI のライブラリをインストール
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
from google.colab import userdata
# OpenAI のAPIキーを取得
# ※環境変数としてOpenAIのAPIキーを設定してください
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
# クライアントの準備
client = OpenAI()

In [ ]:
from typing import List

# テキストデータのベクトル化(embedding)関数
def get_embeddings(texts: List[str]) -> List[List[float]]:

  response = client.embeddings.create(
    input=texts,
    model="text-embedding-ada-002"
 )
  return response

In [ ]:
# ファイルパスからファイルを取得する
def get_file_contents(file_paths):
    file_contents = []
    for path in file_paths:
        try:
            with open(path, 'r') as file:
                content = file.read()
                file_contents.append(content)
        except IOError:
            print(f"ファイル {path} の読み込み中にエラーが発生しました。")
    return file_contents

In [ ]:
# ファイルパスのリストを受け取り、
# 各ファイルの内容からOpenAIのAPIを使用して埋め込みベクトルを取得し、
# 全ファイルの埋め込みベクトルをリストで返す関数
def get_all_embeddings(filePaths):
  embeddings = []
  for filePath in filePaths:
    # 8000トークン以内のテキストコンテンツの取得
    file_contents = get_file_contents(filePath)
    # embeddingの実行
    response = get_embeddings(file_contents)
    batch_embeddings = [record.embedding for record in response.data]
    # ベクトルデータをまとめる
    embeddings.extend(batch_embeddings)
  return embeddings

In [ ]:
import numpy as np

# npyファイルとして保存する関数
def save_embeddings(embeddings, file_path):
    np.save(file_path, embeddings)

In [ ]:
# numpy配列に変換
target_embeds = get_all_embeddings(file_path_groups)
target_embeds = np.array(target_embeds).astype("float32")

# Embeddingを保存
save_embeddings(target_embeds,"embeddings_all.npy")

# ベクトルデータベースへのIndex登録


In [ ]:
# Faissパッケージのインストール
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 2.0 MB/s eta 0:00:00


In [ ]:
# ベクトルデータのファイルパス
embedding_file =  './embeddings_all.npy'

In [ ]:
# ファイルからEmbeddingを読み込む
target_embeds = np.load(embedding_file)

# インデックス数の取得
d = target_embeds.shape[1]
print(d)

# Faissのインデックス生成
# ここで利用している検索アルゴリズムはIndexFlatL2(L2ノルム)で、ユークリッド距離を使用してベクトル間の類似を計算する
# 他にも IndexFlatIP(コサイン類似度）、IndexIVFFlat(高速化アルゴリズム)を利用可能
index = faiss.IndexFlatL2(d)

# 対象テキストをインデックスに追加
index.add(target_embeds)

# 作業対象のファイルを読み込み、作業候補者を出す

In [ ]:
# テキストを読み込む
txt_file_path = "./{作業対象のファイル}.txt"

with open(txt_file_path, "r", encoding="utf-8") as file:
    in_text = file.read()

FileNotFoundError: [Errno 2] No such file or directory: './{作業対象のファイル}.txt'

In [ ]:
# 作業対象のファイルをベクトル化する
response =client.embeddings.create(
    input=in_text,
    model="text-embedding-ada-002"
)

# ベクトル化したデータをnumpy配列に変換
in_embeds = [record.embedding for record in response.data]
in_embeds = np.array(in_embeds).astype("float32")

NameError: name 'client' is not defined

In [ ]:
import csv

# 作業推薦者を割り出す関数
def recommend_workers(query_embedding, csv_file, index, k=10):
    # 近傍探索の実行
    D, I = index.search(query_embedding, k)
    print(D)
    print(I)

    # 類似ファイルのインデックス値を取得
    similar_indices = I[0].tolist()

    # CSVファイルから既存のファイルパスと担当者IDを読み込む
    file_paths = []
    worker_ids = []
    with open(csv_file, mode='r') as file:
        reader = csv.reader(file)
        next(reader)  # ヘッダーをスキップ
        for row in reader:
            file_paths.append(row[1])
            worker_ids.append(int(row[2]))

    # 類似ファイルの担当者IDを取得
    similar_worker_ids = [worker_ids[i] for i in similar_indices]

    # 担当者IDの出現回数をカウント
    worker_id_counts = {}
    for worker_id in similar_worker_ids:
        worker_id_counts[worker_id] = worker_id_counts.get(worker_id, 0) + 1

    # 出現回数の多い順に担当者IDを取得
    sorted_worker_ids = sorted(worker_id_counts, key=worker_id_counts.get, reverse=True)

    # 推薦する作業担当者のIDを取得
    recommended_worker_ids = sorted_worker_ids[:3]  # 上位3人を推薦

    return recommended_worker_ids

In [ ]:
# 作業候補者を出す関数の実行
recommended_worker_ids = recommend_workers(in_embeds, csv_file, index)

# 例えば以下のような形で出力される 上からベクトルの距離、類似ファイルのインデックス、推薦された作業者ID
# [[0.03189817 0.04887741 0.08837742 0.11160477 0.13243756 0.1446165 0.18959308 0.22103558 0.24178305 0.25575137]]
# [[458 459 475 492 487 483 484 497 462 469]]
# Recommended worker IDs: [5, 3, 4]

print("Recommended worker IDs:", recommended_worker_ids)

NameError: name 'recommend_workers' is not defined